In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import welch, hann

In [2]:
def get_rfft(data, times, sample_rate):
    return np.fft.rfftfreq(data.size, d=1/sample_rate), np.fft.rfft(data, norm="backward")

In [18]:
def generate_time_domain_detector_noise(time_array, noise_amplitude):
    noise = np.random.randn(time_array.size) * noise_amplitude
    return noise

def generate_detector_noise_psd(sample_rate, duration, noise_amplitude):
    deltaT = 1/sample_rate
    times = np.arange(0, duration, deltaT)
    td_noise = generate_time_domain_detector_noise(times, noise_amplitude)
    f, noise_psd = welch(td_noise, fs=sample_rate, nperseg=times.size)
    noise_psd /= 2
    return {'frequencies': f, 'noisePSD': noise_psd}

In [17]:
class Likelihood(object):
    def __init__(self, frequencies, fft_data, detector_noise_psd, duration):
        self.parameters = dict()
        self.fft_data = fft_data
        self.detector_noise_psd = detector_noise_psd
        self.duration = duration
        
    def log_likelihood_function(self):
        resolved_strain = get_resolved_signals(self.parameters)
        unresolved_psd = get_unresolved_psd(self.parameters)
        total_psd = detector_noise_psd + unresolved_psd
        return -np.sum(2 * np.abs(resolved_strain - fft_data)**2 / (self.duration * total_psd)) - np.sum(np.log(np.pi * total_psd * self.duration))
